In [1]:
!export PATH=/usr/lib/jvm/java-8-openjdk-amd64/jre/bin:/u3/cs451/packages/spark/bin:/u3/cs451/packages/hadoop/bin:/u3/cs451/packages/maven/bin:/u3/cs451/packages/scala/bin:$PATH
!export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/jre

In [2]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark.sparkContext)

import html

In [4]:
business_df = spark.read.parquet("Data/yelp_business.parquet")

business_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- business_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- categories: string (nullable = true)



In [5]:
review_df = spark.read.parquet("Data/review_train.parquet")

review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)



In [6]:
user_df = spark.read.parquet("Data/yelp_users.parquet")

user_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- yelping_since: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- fans: long (nullable = true)
 |-- average_stars: double (nullable = true)



In [7]:
# Split the friends_id on comma and create new record for every user_id - friend id pair
# df = friends_df.toPandas()
# df_2 = pd.concat([pd.Series(row['user_id'], row['friend_id'].split(','))              
#                     for _, row in df.iterrows()]).reset_index()
# df_2.columns = ["friends","user_id"]
# df_2 = df_2[["user_id","friends"]]

# pd.io.parquet.get_engine('auto')
# df_2.to_parquet("Data/yelp_friend.parquet")

In [8]:
friends_df = spark.read.parquet("Data/yelp_friends.parquet")
friends_df = friends_df.select(col("user_id"),trim(col("friends")))
friends_df = friends_df.withColumnRenamed("trim(friends)","friends")
friends_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- friends: string (nullable = true)



In [9]:
business_df.createOrReplaceTempView("business")
user_df.createOrReplaceTempView("user")
review_df.createOrReplaceTempView("review")
friends_df.createOrReplaceTempView("friends")

In [10]:
# new_user_df = spark.read.parquet("Data/users.parquet")
# new_user_df.printSchema()
# new_user_df.createOrReplaceTempView("new_user")
# q = """select user_id, friends from new_user where user_id = 'uxKSnOVAoEj4I6X9YhLBlg' """
# ans = sqlContext.sql(q)
# df_q = ans.toPandas()
# print(df_q.iloc[0,1])

In [11]:
# u_id = 'uxKSnOVAoEj4I6X9YhLBlg'

# q = """select user_id, trim(friends) as friends from friends where user_id = 'uxKSnOVAoEj4I6X9YhLBlg' """

# ans = sqlContext.sql(q)
# # ans = ans.select(col("user_id") ,trim(col("friend_id")))
# ans.show(10)

In [12]:
u_id = 'om5ZiponkpRqUNa3pVPiRg'

query = """
select business_id, count(*) as 4_5_stars_count 
from review
where user_id in
    (select trim(f.friends) from friends f
    inner join user u on trim(f.friends)= u.user_id
    where f.user_id = "{}") 
and stars >= 4
and business_id not in (select business_id from review where user_id = "{}")
group by business_id
order by count(*) desc limit 100
""".format(u_id, u_id)

friend_recoms_df = sqlContext.sql(query)

friend_recoms_df.toPandas()

,business_id,4_5_stars_count
0,nT16Y6AsJDwEpUB1JICKzg,20
1,dc3uoAmNo5STqKV6mlD_aA,14
2,JmZj7wzAJ7_4ksjG9WXdqw,13
3,pfpOi3Q-Yap72z2bLKdWPQ,12
4,nu-DHeDtxnOW5vYj93dGPQ,8
5,JMiaNitMzMbJm6Kh0RbT5A,8
6,mG71gQ5HWl7ut9OCN6NJKA,8
7,pZ1quwXnYozLIZIVABojhw,8
8,K6XIGkyk7-fuOQtA8i7p6A,8
9,N93EYZy9R0sdlEvubu94ig,8


In [13]:
friend_recoms_df = friend_recoms_df.sample(False, 0.5).limit(10)
friend_recoms_df.toPandas()

,business_id,4_5_stars_count
0,dc3uoAmNo5STqKV6mlD_aA,14
1,N93EYZy9R0sdlEvubu94ig,8
2,pZ1quwXnYozLIZIVABojhw,8
3,nu-DHeDtxnOW5vYj93dGPQ,8
4,K6XIGkyk7-fuOQtA8i7p6A,8
5,RtUvSWO_UZ8V3Wpj0n077w,7
6,5r6-G9C4YLbC7Ziz57l3rQ,7
7,O-uIEuv7JLUHajkemx_sVw,7
8,3qTSOuJ-ZyU1L7o62KHSIg,6
9,qaNt4vtVdge_S68DVjw5Jg,6


In [14]:
def getBusinessDetails(in_bus):
    
    a = in_bus.alias("a")
    b = business_df.alias("b")
    
    return a.join(b, col("a.business_id") == col("b.business_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.business_name'),col('b.categories'),
                                                           col('b.stars'),col('b.review_count'),
                                                           col('b.latitude'),col('b.longitude')])

In [15]:
def getFriendRecoms(u_id, sim_bus_limit=10):
    
    query = """
    select business_id, count(*) as 4_5_stars_count 
    from review
    where user_id in
        (select f.friends from friends f
        inner join user u on f.friends = u.user_id
        where f.user_id = "{}") 
    and stars >= 4 
    and business_id not in (select business_id from review where user_id = "{}")
    group by business_id
    order by count(*) desc
    """.format(u_id, u_id)

    friend_recoms_df = sqlContext.sql(query)
    
    #friend_recoms_df = friend_recoms_df.sample(False, 0.5).limit(sim_bus_limit)

    return getBusinessDetails(friend_recoms_df)

In [16]:
# test recoms for a user

u_id = 'uxKSnOVAoEj4I6X9YhLBlg'

friend_recom_df = getFriendRecoms(u_id)

print("Businesses recommended to user by best useful friends:")
friend_recom_df.toPandas().head(20)

Businesses recommended to user by best useful friends:


,business_id,4_5_stars_count,business_name,categories,stars,review_count,latitude,longitude
0,SjgeuBlgKER9yegpoxT99w,7,Nomé Izakaya,"Tapas Bars, Japanese, Restaurants, Lounges, Ba...",4.0,512,43.762650,-79.411469
1,kOFDVcnj-8fd3doIpCQ06A,7,Mildred's Temple Kitchen,"Venues & Event Spaces, Event Planning & Servic...",4.0,706,43.639911,-79.420424
2,SGP1jf6k7spXkgwBlhiUVw,5,Kekou Gelato House,"Ice Cream & Frozen Yogurt, Food, Desserts, Caf...",4.5,411,43.655983,-79.392686
3,k6zmSLmYAquCpJGKNnTgSQ,5,The Stockyards,"American (Traditional), Barbeque, Restaurants,...",4.0,693,43.681346,-79.426147
4,0a2O150ytxrDjDzXNfRWkA,4,Miku,"Sushi Bars, Restaurants, Japanese, Seafood",4.0,604,43.641235,-79.377370
5,G6EkDTXZ6zMUovg7JTG4YQ,4,Vietnam Noodle Star,"Noodles, Restaurants, Vietnamese",3.5,183,43.804583,-79.287792
6,RwRNR4z3kY-4OsFqigY5sw,4,Uncle Tetsu's Japanese Cheesecake,"Bakeries, Japanese, Restaurants, Desserts, Food",3.5,939,43.655969,-79.384013
7,GcxE5hK_TaHP4EZFDYz2mg,3,Copacabana Brazilian Steakhouse,"Nightlife, Steakhouses, Bars, Restaurants, Bar...",3.5,275,43.648344,-79.387831
8,8I5U8OQ06nSxX2y4PPOWzQ,3,Japango,"Restaurants, Sushi Bars",3.5,643,43.655066,-79.385297
9,-J6FVdY9pSgAdFmmalO-pQ,3,Jatujak,"Food, Restaurants, Thai, Food Delivery Services",4.0,273,43.736087,-79.307939
